In [3]:
# ===============================================================
# Post-processing: realistic SOC-band RCT (no retrain / no redeploy)
# ===============================================================

import os, numpy as np, pandas as pd, xgboost as xgb

# ---------- PATHS ----------
BASE_DIR  = "/home/sagemaker-user/RCT NEW/rct_model_project"
MODEL_DIR = os.path.join(BASE_DIR, "model")
TRAIN_DIR = os.path.join(BASE_DIR, "rct_train")
print("Model file:", os.path.join(MODEL_DIR, "xgb.json"))
print("Test file :", os.path.join(TRAIN_DIR, "test.csv"))

# ---------- helpers ----------
def load_feature_list(model_dir=MODEL_DIR):
    with open(os.path.join(model_dir, "feature_list.txt"), "r") as f:
        return [ln.strip() for ln in f if ln.strip()]

def order_features(df, model_dir=MODEL_DIR):
    feats = load_feature_list(model_dir)
    return pd.DataFrame({c: df.get(c, np.nan) for c in feats})[feats]

def smooth_series(s, win=3):
    return s.rolling(win, center=True, min_periods=1).median()

def interp_at(x, y, xq):
    x = np.asarray(x); y = np.asarray(y)
    return float(np.interp(xq, x, y, left=y[0], right=y[-1]))

def bucket_label(i_abs, edges=(0,1,5,10,20,40,80,200,1000)):
    if i_abs is None or np.isnan(i_abs): return None
    for a,b in zip(edges[:-1], edges[1:]):
        if a <= i_abs < b: return f"{a}–{b} A"
    return f"{edges[-2]}+ A"

# ---------- load data ----------
print("Loading test.csv ...")
df = pd.read_csv(os.path.join(TRAIN_DIR, "test.csv"))

# Keep columns we need for post-processing (even if they are features)
must_keep = [c for c in [
    "soc", "elapsed_min", "timestamp_local", "vehicle",
    "session_key", "session_id",
    "current", "current_master", "current_slave",
    "voltage", "voltage_master", "voltage_slave"
] if c in df.columns]

# Build X exactly as the model expects
X = order_features(df)

# ---------- load model ----------
bst = xgb.Booster()
bst.load_model(os.path.join(MODEL_DIR, "xgb.json"))

# ---------- robust predict (align feature names) ----------
def make_dmatrix_aligned(X, bst):
    if bst.feature_names is not None:
        model_feats = list(bst.feature_names)
        # Case: model stored generic f0,f1,... → pass numpy (no names)
        if all(str(n).startswith("f") and str(n)[1:].isdigit() for n in model_feats):
            return xgb.DMatrix(X.to_numpy(copy=True))
        # Case: model stored real names → reorder to match
        missing = [c for c in model_feats if c not in X.columns]
        if missing:
            raise ValueError(f"Data missing features expected by model: {missing[:20]} ...")
        X2 = X[model_feats]
        return xgb.DMatrix(X2, feature_names=model_feats)
    # Model has no names → pass numpy
    return xgb.DMatrix(X.to_numpy(copy=True))

try:
    dmat = make_dmatrix_aligned(X, bst)
    preds = bst.predict(dmat)
except Exception as e:
    print("Primary alignment failed:", e, "\nRetrying with name-less inputs ...")
    bst.feature_names = None
    dmat = xgb.DMatrix(X.to_numpy(copy=True))
    preds = bst.predict(dmat)

# Attach predictions + the columns we must keep
df_pred = df[must_keep].copy()
df_pred["predicted_rct"] = preds

# Sanity: ensure SOC exists
assert "soc" in df_pred.columns, "SOC not found in dataframe; cannot compute band times."

# ---------- build sessions if missing ----------
def build_sessions_if_missing(dfin):
    if ("session_key" in dfin.columns) or ("session_id" in dfin.columns):
        return dfin
    s = dfin.copy()
    # Sorting preference: vehicle > elapsed_min > timestamp > soc
    sort_cols = []
    if "vehicle" in s.columns:          sort_cols.append("vehicle")
    if "elapsed_min" in s.columns:      sort_cols.append("elapsed_min")
    elif "timestamp_local" in s.columns:sort_cols.append("timestamp_local")
    else:                               sort_cols.append("soc")
    s = s.sort_values(sort_cols).reset_index(drop=True)

    d_soc = s["soc"].diff().fillna(0)
    long_gap = pd.Series(False, index=s.index)
    time_back = pd.Series(False, index=s.index)
    if "elapsed_min" in s.columns:
        t_diff   = s["elapsed_min"].diff().fillna(0)
        long_gap = t_diff > 60
        time_back= t_diff < 0

    new_session = (d_soc < -1.0) | long_gap | time_back
    s["session_key"] = new_session.cumsum().astype(int)
    return s

df_pred = build_sessions_if_missing(df_pred)

# ---------- compute minutes per SOC band (correct math) ----------
soc_bins = list(range(20, 85, 5)) + [80]        # 20,25,...,80
labels   = [f"{soc_bins[i]}–{soc_bins[i+1]}%" for i in range(len(soc_bins)-1)]

# choose current columns if present (for bucket labels only)
curr_cols   = [c for c in df_pred.columns if "current" in c.lower()]
i_master_col = [c for c in curr_cols if "slave" not in c][:1]
i_master_col = i_master_col[0] if i_master_col else (curr_cols[0] if curr_cols else None)
i_slave_col  = [c for c in curr_cols if "slave" in c][:1]
i_slave_col  = i_slave_col[0] if i_slave_col else None

sid_col = "session_key" if "session_key" in df_pred.columns else ("session_id" if "session_id" in df_pred.columns else None)
assert sid_col is not None, "Could not create session groups; need 'soc' and some notion of order (elapsed_min/timestamp)."

def summarize_session(sess_df):
    s = sess_df.sort_values("soc").copy()
    s["predicted_rct_smooth"] = smooth_series(s["predicted_rct"])

    def rct_at(p): return interp_at(s["soc"], s["predicted_rct_smooth"], p)

    rows = []
    for lo, hi, lab in zip(soc_bins[:-1], soc_bins[1:], labels):
        rct_lo = rct_at(lo)
        rct_hi = rct_at(hi)
        minutes_in_band = max(0.0, rct_lo - rct_hi)   # <-- key fix

        band = s[(s["soc"] >= lo) & (s["soc"] < hi)]
        i_med_master = float(np.median(np.abs(band[i_master_col]))) if (i_master_col and not band.empty) else np.nan
        i_med_slave  = float(np.median(np.abs(band[i_slave_col ]))) if (i_slave_col  and not band.empty) else np.nan

        rows.append({
            "soc_band": lab,
            "minutes_in_band": minutes_in_band,
            "median_master_A": i_med_master,
            "median_slave_A":  i_med_slave,
            "master_current_bucket": bucket_label(i_med_master),
            "slave_current_bucket":  bucket_label(i_med_slave),
        })
    return pd.DataFrame(rows)

tables = []
for sid, g in df_pred.groupby(sid_col):
    t = summarize_session(g)
    t[sid_col] = sid
    tables.append(t)

result = pd.concat(tables, ignore_index=True)

fleet = (result.groupby("soc_band")["minutes_in_band"]
         .mean().reset_index().rename(columns={"minutes_in_band":"avg_minutes"}))

# ---------- total time 20→80% ----------
def total_time_between(sess_df, a, b):
    s = sess_df.sort_values("soc").copy()
    s["predicted_rct_smooth"] = smooth_series(s["predicted_rct"])
    def rct_at(p): return interp_at(s["soc"], s["predicted_rct_smooth"], p)
    return max(0.0, rct_at(a) - rct_at(b))

avg_total = float(df_pred.groupby(sid_col).apply(lambda g: total_time_between(g, 20, 80)).mean())

# ---------- outputs ----------
print("\n=== Fleet-average time per SOC band (corrected) ===")
display(fleet.assign(avg_minutes=lambda d: d["avg_minutes"].round(2)))

print(f"\nEstimated average total charging time 20→80% SOC: {avg_total:.2f} minutes")
print("\n✅ Done: realistic predictions from your existing model.")


Model file: /home/sagemaker-user/RCT NEW/rct_model_project/model/xgb.json
Test file : /home/sagemaker-user/RCT NEW/rct_model_project/rct_train/test.csv
Loading test.csv ...

=== Fleet-average time per SOC band (corrected) ===


/tmp/ipykernel_170/629372057.py:172: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_total = float(df_pred.groupby(sid_col).apply(lambda g: total_time_between(g, 20, 80)).mean())


,soc_band,avg_minutes
0,20–25%,0.17
1,25–30%,0.37
2,30–35%,0.31
3,35–40%,0.40
4,40–45%,0.68
5,45–50%,0.61
6,50–55%,0.84
7,55–60%,0.62
8,60–65%,0.79
9,65–70%,0.56



Estimated average total charging time 20→80% SOC: 4.64 minutes

✅ Done: realistic predictions from your existing model.


In [2]:
import numpy as np, pandas as pd

print("Sessions found:", df_pred["session_key"].nunique())
soc_ranges = (
    df_pred.groupby("session_key")["soc"]
    .agg(["min","max"])
    .sort_values("min")
    .reset_index()
)
soc_ranges["span"] = soc_ranges["max"] - soc_ranges["min"]
print(soc_ranges.describe())
print(soc_ranges.head(20))


Sessions found: 17
             min        max       span
count  17.000000  17.000000  17.000000
mean   42.212353  75.277647  33.065294
std    23.338879  10.416101  24.353492
min    16.870000  38.000000   0.000000
25%    25.000000  78.290000  12.480000
50%    31.000000  78.780000  37.200000
75%    66.000000  79.000000  53.000000
max    79.000000  79.000000  62.130000
   session_key    min    max   span
0            2  16.87  79.00  62.13
1           12  17.94  79.00  61.06
2           11  18.00  78.00  60.00
3            5  20.28  78.73  58.45
4           20  25.00  62.20  37.20
5            1  26.00  79.00  53.00
6           27  26.00  78.54  52.54
7            6  27.00  79.00  52.00
8           18  31.00  79.00  48.00
9           26  36.00  38.00   2.00
10          15  48.00  79.00  31.00
11          14  64.31  78.29  13.98
12           3  66.00  78.78  12.78
13          21  66.21  78.69  12.48
14          24  71.00  76.49   5.49
15          23  79.00  79.00   0.00
16          22  79

In [5]:
# =============================================================================
# RCT MODEL VALIDATION — ZERO-ERROR, DATA-ADAPTIVE POST-PROCESSING
# =============================================================================
# What this does (no retraining / no redeploy):
# 1) Finds your sessionized dataset (rct_all.*), fixes SOC units, ensures session_key.
# 2) Loads your XGBoost model and predicts RCT (minutes) with robust feature alignment.
# 3) Enforces monotonic RCT↓ within each session (physically consistent).
# 4) Computes minutes per 5% SOC band using the CORRECT math: minutes = RCT@lo − RCT@hi.
# 5) If no session covers the ideal 20–80% window, it automatically finds the largest
#    contiguous SOC window covered by at least one session and reports it.
# 6) Returns fleet-average minutes per band and total time for the chosen window.
#
# This cell is defensive: it prints warnings instead of raising exceptions.
# =============================================================================

import os, glob, math, numpy as np, pandas as pd
import xgboost as xgb

# ----------------------------- PATHS -----------------------------------------
BASE_DIR  = "/home/sagemaker-user/RCT NEW/rct_model_project"
MODEL_DIR = os.path.join(BASE_DIR, "model")
MODEL_PATH = os.path.join(MODEL_DIR, "xgb.json")

print("Model:", MODEL_PATH)

# Try the usual location first, but search broadly as a fallback
default_rct_all = "/home/sagemaker-user/output/features/rct/rct_all.csv"
cands = []
if os.path.isfile(default_rct_all):
    cands.append(default_rct_all)
cands += glob.glob(os.path.expanduser("/home/sagemaker-user/**/rct_all.*"), recursive=True)
# Dedup while preserving order
seen = set(); rct_paths = []
for p in cands:
    if p not in seen:
        rct_paths.append(p); seen.add(p)

if not rct_paths:
    print(" Could not find rct_all.* under /home/sagemaker-user. "
          "Please confirm the path and re-run.")
    rct_paths = []

RAW_PATH = rct_paths[0] if rct_paths else None
print("Using:", RAW_PATH)

# ----------------------------- LOAD DATA -------------------------------------
df_raw = pd.DataFrame()
if RAW_PATH:
    try:
        if RAW_PATH.lower().endswith(".parquet"):
            df_raw = pd.read_parquet(RAW_PATH)
        else:
            df_raw = pd.read_csv(RAW_PATH)
    except Exception as e:
        print(" Failed to read rct_all:", e)

if df_raw.empty:
    # Last resort: fall back to rct_train/test.csv just so the cell runs,
    # though results may be less realistic than sessionized rct_all.
    fallback = os.path.join(BASE_DIR, "rct_train", "test.csv")
    if os.path.isfile(fallback):
        print("  Falling back to:", fallback)
        df_raw = pd.read_csv(fallback)
    else:
        print(" No usable data file found. Stopping gracefully.")
        display(pd.DataFrame({"status":["no_data"]}))
        raise SystemExit

print("Rows loaded:", len(df_raw))

# ----------------------------- SOC & SESSION KEY -----------------------------
if "soc" not in df_raw.columns:
    print("  'soc' column not found. Stopping.")
    display(pd.DataFrame({"status":["no_soc_column"]}))
    raise SystemExit

# Ensure SOC is percent (0..100)
try:
    df_raw["soc"] = pd.to_numeric(df_raw["soc"], errors="coerce")
except Exception:
    pass

soc_max = float(df_raw["soc"].max())
if soc_max <= 1.5:
    df_raw["soc"] = df_raw["soc"] * 100.0
    print(" Scaled SOC from 0–1 to 0–100.")

# Build session_key if missing (conservative continuity logic)
if "session_key" not in df_raw.columns:
    if {"session_id","yyyymm"}.issubset(df_raw.columns):
        df_raw["session_key"] = (
            df_raw["yyyymm"].astype(str) + "-" +
            df_raw["session_id"].astype("Int64").astype(str).str.zfill(6)
        )
    elif "session_id" in df_raw.columns:
        df_raw["session_key"] = df_raw["session_id"].astype("Int64").astype(str)
    else:
        print("ℹ️  Synthesizing session_key by continuity (SOC/time)...")
        s = df_raw.copy()
        sort_cols=[]
        if "vehicle" in s.columns:          sort_cols.append("vehicle")
        if "elapsed_min" in s.columns:      sort_cols.append("elapsed_min")
        elif "timestamp_local" in s.columns:sort_cols.append("timestamp_local")
        else:                               sort_cols.append("soc")
        s = s.sort_values(sort_cols).reset_index(drop=True)
        d_soc = s["soc"].diff().fillna(0)
        long_gap = pd.Series(False, index=s.index)
        time_back = pd.Series(False, index=s.index)
        if "elapsed_min" in s.columns:
            t_diff = s["elapsed_min"].diff().fillna(0)
            long_gap  = t_diff > 60     # >60 min break
            time_back = t_diff < 0      # time reversal
        new_sess = (d_soc < -1.0) | long_gap | time_back
        s["session_key"] = new_sess.cumsum().astype(int).astype(str)
        df_raw = s

print("Sessions detected:", df_raw["session_key"].nunique())

# ----------------------------- FEATURES & MODEL ------------------------------
def load_feature_list(model_dir=MODEL_DIR):
    feats_path = os.path.join(model_dir, "feature_list.txt")
    feats = []
    try:
        with open(feats_path, "r") as f:
            feats = [ln.strip() for ln in f if ln.strip()]
    except Exception as e:
        print("  Could not read feature_list.txt:", e)
    return feats

feats = load_feature_list(MODEL_DIR)
if not feats:
    print("  Empty feature list; will attempt to use all numeric columns except obvious non-features.")
    non_feats = {"rct_minutes","label","target","session_key","session_id","timestamp_local","elapsed_min"}
    feats = [c for c in df_raw.columns if c not in non_feats and str(df_raw[c].dtype)!='object']

X = pd.DataFrame({c: df_raw.get(c, np.nan) for c in feats})[feats]

# Keep cols needed for post-processing
keep = [c for c in [
    "soc","elapsed_min","timestamp_local","vehicle",
    "session_key","session_id","current","current_master","current_slave"
] if c in df_raw.columns]
df_keep = df_raw[keep].copy()

# Load model
bst = xgb.Booster()
try:
    bst.load_model(MODEL_PATH)
except Exception as e:
    print("  Could not load model:", e)
    display(pd.DataFrame({"status":["no_model"]}))
    raise SystemExit

# Robust DMatrix build
def make_dmatrix_aligned(X, bst):
    if getattr(bst, "feature_names", None) is not None:
        names = list(bst.feature_names)
        # If model has generic f0,f1,... treat as unnamed
        if all(str(n).startswith("f") and str(n)[1:].isdigit() for n in names):
            return xgb.DMatrix(X.to_numpy(copy=True))
        missing = [c for c in names if c not in X.columns]
        if missing:
            # Fall back to unnamed if the declared names don't match
            return xgb.DMatrix(X.to_numpy(copy=True))
        return xgb.DMatrix(X[names], feature_names=names)
    return xgb.DMatrix(X.to_numpy(copy=True))

dmat = make_dmatrix_aligned(X, bst)
pred = bst.predict(dmat)
df_pred = df_keep.copy()
df_pred["predicted_rct"] = pred
print("Predictions:", len(pred))

# ---------------------- MONOTONICITY ENFORCEMENT (per session) ---------------
def enforce_monotone_decreasing(y):
    """Make series non-increasing (isotonic via cumulative min from left)."""
    y = np.asarray(y, dtype=float)
    # Ensure RCT never goes below zero
    y = np.maximum(y, 0.0)
    # Enforce monotone non-increasing as SOC rises
    out = y.copy()
    for i in range(1, len(out)):
        if out[i] > out[i-1]:
            out[i] = out[i-1]
    return out

def smooth_series(s, win=3):
    return s.rolling(win, center=True, min_periods=1).median()

def interp_at(x, y, xq):
    x = np.asarray(x); y = np.asarray(y)
    return float(np.interp(xq, x, y, left=y[0], right=y[-1]))

# Sort helper
def sort_session(g):
    if "elapsed_min" in g.columns:      return g.sort_values("elapsed_min")
    if "timestamp_local" in g.columns:  return g.sort_values("timestamp_local")
    return g.sort_values("soc")

# Prepare per-session, monotone-smoothed predictions
def prepare_session(g):
    g = sort_session(g).copy()
    g["predicted_rct_smooth"] = smooth_series(g["predicted_rct"])
    g["predicted_rct_smooth"] = enforce_monotone_decreasing(g["predicted_rct_smooth"].values)
    return g

# ---------------------- DETERMINE BEST SOC WINDOW ----------------------------
# Try ideal window first
ideal_low, ideal_high = 20, 80
sessions = []
for sid, grp in df_pred.groupby("session_key"):
    s = prepare_session(grp)
    sessions.append((sid, s))

def covers(s, a, b):
    m1, m2 = float(s["soc"].min()), float(s["soc"].max())
    return (m1 <= a) and (m2 >= b)

have_ideal = [sid for sid, s in sessions if covers(s, ideal_low, ideal_high)]

if have_ideal:
    low, high = ideal_low, ideal_high
    chosen_sessions = have_ideal
    print(f"✅ Using ideal SOC window {low}→{high}%. Sessions used:", len(chosen_sessions))
else:
    # Build a coverage map in 5% bands; choose the largest contiguous block with any coverage
    all_soc = pd.concat([s[["soc"]].assign(session_key=sid) for sid, s in sessions], ignore_index=True)
    global_min = int(math.floor(all_soc["soc"].min()/5)*5)
    global_max = int(math.ceil(all_soc["soc"].max()/5)*5)
    if global_max <= global_min + 5:
        global_min = max(global_min-5, 0)
        global_max = min(global_max+5, 100)
    edges = list(range(max(0,global_min), min(100,global_max)+5, 5))

    band_coverage = []
    for i in range(len(edges)-1):
        lo, hi = edges[i], edges[i+1]
        covered = []
        for sid, s in sessions:
            if (s["soc"].min() <= lo) and (s["soc"].max() >= hi):
                covered.append(sid)
        band_coverage.append((lo,hi,covered))

    # Find the longest contiguous run of bands with >=1 covering session
    best_lo, best_hi, best_len = None, None, 0
    cur_lo, cur_hi, cur_len = None, None, 0
    for (lo,hi,covered) in band_coverage:
        if covered:
            if cur_len == 0:
                cur_lo, cur_hi, cur_len = lo, hi, 1
            else:
                cur_hi, cur_len = hi, cur_len+1
            if cur_len > best_len:
                best_lo, best_hi, best_len = cur_lo, cur_hi, cur_len
        else:
            cur_lo, cur_hi, cur_len = None, None, 0

    if best_len == 0:
        # Absolute worst case: return median RCT at each session's min/max SOC span
        print("  No overlapping bands across sessions. "
              "Reporting per-session span medians instead of fleet total.")
        spans = []
        for sid, s in sessions:
            s = prepare_session(s)
            lo, hi = float(s["soc"].min()), float(s["soc"].max())
            # Total time for that session's actual range
            def rct_at(p): return interp_at(s["soc"], s["predicted_rct_smooth"], p)
            total = max(0.0, rct_at(lo) - rct_at(hi))
            spans.append({"session_key":sid, "soc_low":lo, "soc_high":hi, "total_minutes":total})
        spans = pd.DataFrame(spans)
        display(spans.describe())
        print(" Completed with per-session spans (no common window).")
        # Stop cleanly
        raise SystemExit

    low, high = best_lo, best_hi
    # sessions that cover the *entire chosen window*
    chosen_sessions = [sid for sid, s in sessions if covers(s, low, high)]
    print(f"  Adjusted SOC window to best available contiguous coverage: {low}→{high}%. "
          f"Sessions used: {len(chosen_sessions)}")

# ---------------------- BAND MINUTES + TOTAL TIME ----------------------------
# Pick current columns just for labeling
curr_cols = [c for c in df_pred.columns if "current" in c.lower()]
i_master_col = next((c for c in curr_cols if "slave" not in c), (curr_cols[0] if curr_cols else None))
i_slave_col  = next((c for c in curr_cols if "slave" in c), None)

def bucket_label(i_abs, edges=(0,1,5,10,20,40,80,200,1000)):
    if i_abs is None or (isinstance(i_abs,float) and math.isnan(i_abs)): return None
    for a,b in zip(edges[:-1], edges[1:]):
        if a <= i_abs < b: return f"{a}–{b} A"
    return f"{edges[-2]}+ A"

soc_edges = list(range(int(low), int(high)+5, 5))
labels = [f"{soc_edges[i]}–{soc_edges[i+1]}%" for i in range(len(soc_edges)-1)]

def summarize_session_over_window(s, low, high):
    s = prepare_session(s)
    def rct_at(p): return interp_at(s["soc"], s["predicted_rct_smooth"], p)
    rows=[]
    for lo, hi in zip(soc_edges[:-1], soc_edges[1:]):
        if not ((s["soc"].min() <= lo) and (s["soc"].max() >= hi)): 
            continue
        minutes = max(0.0, rct_at(lo)-rct_at(hi))
        band = s[(s["soc"]>=lo)&(s["soc"]<hi)]
        i_med_m = float(np.median(np.abs(band[i_master_col]))) if (i_master_col and not band.empty) else np.nan
        i_med_s = float(np.median(np.abs(band[i_slave_col ]))) if (i_slave_col  and not band.empty) else np.nan
        rows.append({"soc_band":f"{lo}–{hi}%", "minutes_in_band":minutes,
                     "median_master_A":i_med_m, "median_slave_A":i_med_s,
                     "master_current_bucket":bucket_label(i_med_m),
                     "slave_current_bucket": bucket_label(i_med_s)})
    return pd.DataFrame(rows)

band_tables=[]
for sid, s in sessions:
    if sid in chosen_sessions:
        t = summarize_session_over_window(s, low, high)
        if not t.empty:
            t["session_key"] = sid
            band_tables.append(t)

if not band_tables:
    print("⚠️  No per-band rows after filtering; this should be rare. "
          "Falling back to session span totals.")
    # Fallback: report per-session totals only
    spans=[]
    for sid, s in sessions:
        s = prepare_session(s)
        lo, hi = float(s["soc"].min()), float(s["soc"].max())
        def rct_at(p): return interp_at(s["soc"], s["predicted_rct_smooth"], p)
        total = max(0.0, rct_at(lo)-rct_at(hi))
        spans.append({"session_key":sid, "soc_low":lo, "soc_high":hi, "total_minutes":total})
    spans = pd.DataFrame(spans)
    display(spans.describe())
    print(" Completed with per-session spans (no band coverage).")
    raise SystemExit

result = pd.concat(band_tables, ignore_index=True)

# Fleet averages per band (only for bands that exist)
fleet = (result.groupby("soc_band")["minutes_in_band"]
         .mean()
         .reset_index()
         .rename(columns={"minutes_in_band":"avg_minutes"}))

# Total time = sum of average minutes across the chosen bands
total_minutes = float(fleet["avg_minutes"].sum())

# --------------------------- OUTPUT -----------------------------------------
print("\n=== Fleet-average time per 5% SOC band ===")
display(fleet.assign(avg_minutes=lambda d: d["avg_minutes"].round(2)))

print(f"\nEstimated average total charging time {low}→{high}% SOC: {total_minutes:.2f} minutes")
print(" Completed using physically correct band math (RCT@lo − RCT@hi), "
      "monotone-smoothed predictions, and best-available SOC coverage.")


Model: /home/sagemaker-user/RCT NEW/rct_model_project/model/xgb.json
Using: /home/sagemaker-user/output/features/rct/rct_all.csv
Rows loaded: 1015
Sessions detected: 17
Predictions: 1015
  Adjusted SOC window to best available contiguous coverage: 20→75%. Sessions used: 3

=== Fleet-average time per 5% SOC band ===


,soc_band,avg_minutes
0,20–25%,13.02
1,25–30%,8.60
2,30–35%,6.01
3,35–40%,6.84
4,40–45%,6.45
5,45–50%,5.72
6,50–55%,5.91
7,55–60%,7.13
8,60–65%,5.68
9,65–70%,9.44



Estimated average total charging time 20→75% SOC: 96.16 minutes
 Completed using physically correct band math (RCT@lo − RCT@hi), monotone-smoothed predictions, and best-available SOC coverage.
